In [3]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import constants
import extension_tools
import os

from dataproc import build_vocab

# import datasets
# import log_reg
# from dataproc import extract_wvs
# from dataproc import get_discharge_summaries
# from dataproc import concat_and_split
# from dataproc import word_embeddings

# from nltk.tokenize import RegexpTokenizer
# import pandas as pd
# from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We assume to start that you have prepared the MIMIC-III note files/data splits based on the CAML preprocessing notebook, available here:
https://github.com/jamesmullenbach/caml-mimic/blob/master/notebooks/dataproc_mimic_III.ipynb

and have the files in the following directory structure:

#### Intermediate step: write each note to its own file:

In [ ]:
#TOINSERT

### Step one: run cTAKES to perform concept annotation on the notefiles:

In [ ]:
#TOINSERT

### Step two: extract annotated concepts from the XMI files for each note and write to csv, by dev/test/train splits:

In [ ]:
#TOINSERT

### Step three: build the concept-text alignment matrices for each note:
#### This step also constructs a concept vocabulary file based on concepts with >3 occurrences (in TRAINING data).

#### NOTE: this takes some time to run**

In [ ]:
#example, for SNOMED only:
extension_tools.get_concept_text_alignment(inpt_file='/data/swiegreffe6/NEW_MIMIC/mimic3/train_full.csv', concepts_file='/data/swiegreffe6/NEW_MIMIC/patient_notes/concepts_train_ALL.csv', outpt_file='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/train_patient_concepts_matrix_SNOMED.p', lst_terms=['SNOMEDCT_US'], dir_name='SNOMED')

#or as command, 
    python extension_tools.py get_concept_text_alignment --input-dir '/data/swiegreffe6/NEW_MIMIC/mimic3/train_full.csv' --concepts-dir '/data/swiegreffe6/NEW_MIMIC/patient_notes/concepts_train_ALL.csv' --output-dir='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/train_patient_concepts_matrix_SNOMED.p' --codes-list 'SNOMEDCT_US' --dir-name='SNOMED'

In [4]:
#TODO: this step could use some more work, e.g. overlapping concepts, and hyphenated concepts that can't be mapped to any parent codes later
#consider some normalization here?

### Step 3.5: build the concept vocabulary based on occurences in the concept matrices:

In [ ]:
#TODO: SEPARATE THIS METHOD FROM THE ABOVE**

#also outputs the concept_children file to '/data/swiegreffe6/NEW_MIMIC/extracted_concepts/dir_name/concept_vocab_%s_children.txt' % dir_name (hardcoded)

### Step four: build the child-to-parent code mappings based on the ontology resources we want to use:
#### NOTE this file only has to be created once (as long as complete ontologies included), because it's a dictionary for later lookup**

File directory needed to run this code:

    | constants.DATA_DIR
        | Multi_Level_CCS_2015
            | ccs_multi_dx_tool_2015.csv
            | ccs_multi_pr_tool_2015.csv
        | SnomedCT_USEditionRF2_PRODUCTION_20180301T183000Z 
            | Full
                | Terminology
                    | sct2_Relationship_Full_US1000124_20180301.txt

In [ ]:
dirs_map = extension_tools.get_parent_trees()
#make sure constants.DATA_DIR points to directory holding subfolders of concept mappings keys.
#this method writes 'code_parents.p' to constants.concept_write_dir.

#TODOs/NOTE: this file contains/writes a 'lopsided' SNOMED hierarchy, where for each child we only consider one parent.
#We also have not included RXCLASS mapping file here (yet).

  0%|          | 0/888405 [00:00<?, ?it/s]

('LEN DIRS MAP before SNOMED add:', 19020)


 34%|███▍      | 300170/888405 [00:33<01:05, 9019.54it/s]

### Next, we update the concept vocabulary to include the newly added codes

In [4]:
#Make sure path_to_vocab is the location of the (>3 occurrences) concept vocab generated from the training set.
#This outputs the updated concept vocabulary (plus the higher-level codes) to out_dir/concept_vocab.txt

path_to_vocab = ''
extension_tools.update_vocab(dirs_map, path_to_vocab, out_dir='/data/swiegreffe6/NEW_MIMIC/extracted_concepts/ICD9_RXNORM_SNOMED/ICD9_SNOMED_augmented_concept_vocab.txt')

#or as command:
    #python extension_tools.py update_vocab --dirs-map '/data/swiegreffe6/NEW_MIMIC/extracted_concepts/ICD9_SNOMED/ICD9_SNOMED_code_parents.p' --vocab-file '/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/concept_vocab_SNOMED_children.txt' --output-dir '/data/swiegreffe6/NEW_MIMIC/extracted_concepts/SNOMED/' --load-dirs-map

### TODOS: for the data sparsity experiments, setup a pipeline to recalculate the vocab.txt file based on the reduced train data:

e.g. '/data/swiegreffe6/NEW_MIMIC/mimic3/train_full.csv' and '/data/swiegreffe6/NEW_MIMIC/mimic3/vocab.csv' need to be updated

In [2]:
MIMIC_3_DIR

NameError: name 'MIMIC_3_DIR' is not defined

In [4]:
vocab_min = 3
tr = '/data/swiegreffe6/NEW_MIMIC/mimic3/10_percent/train_full.csv' #TODO: UPDATE THIS LINE FOR EACH DIRECTORY
vname = '/data/swiegreffe6/NEW_MIMIC/mimic3/10_percent/vocab.csv'
build_vocab.build_vocab(vocab_min, tr, vname)

IOError: [Errno 2] No such file or directory: '/data/swiegreffe6/NEW_MIMIC/mimic3/10_percent/train_full.csv'

### and to retrain the word embeddings

### and to rebuild the concept embeddings based off of the new word embeddings

### TODO: (re)calculate word-concept embedding matrix (for both the data sparsity and the concepts change experiments)

### TODOs: pretrain the concept embeddings again when add or remove concepts